
### <span style="color:black"> UNIVERSIDAD DE LOS ANDES

Facultad de Ingeniería

Departamento de ingeniería Industrial

Maestría Inteligencia Analítica para la toma de Decisiones

### <span style="color:black">  MIIA4200_01 



Juan Camilo Angarita Villamizar      200711784

_________________________________

# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [2]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
4662,7.3,0.17,0.36,8.20,0.028,44.0,111.0,0.99272,3.14,0.41,12.4,6,white
5873,7.2,0.41,0.30,2.10,0.083,35.0,72.0,0.99700,3.44,0.52,9.4,5,red
3844,6.7,0.37,0.25,2.50,0.028,24.0,84.0,0.99090,3.14,0.36,11.7,6,white
4121,6.3,0.23,0.22,17.45,0.054,42.0,151.0,0.99853,3.12,0.60,9.3,6,white
3695,6.4,0.15,0.40,1.30,0.053,61.0,146.0,0.99112,3.17,0.68,11.0,6,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [4]:
pd.pivot_table(data,index='quality', columns='type', values='fixed acidity' , aggfunc='count')
#df2.pivot_table(values='X',rows=['Y','Z'],cols='X',aggfunc='count')

type,red,white
quality,,
3,10.0,20.0
4,53.0,163.0
5,681.0,1457.0
6,638.0,2198.0
7,199.0,880.0
8,18.0,175.0
9,NaN,5.0


# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


In [5]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white


In [6]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'type'],
      dtype='object')

In [7]:
names = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']

scaler = preprocessing.StandardScaler()

scaled_df = scaler.fit_transform(data[names])
scaled_df = pd.DataFrame(scaled_df, columns=names)
scaled_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558
1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615
2,0.682458,-0.362438,0.559966,0.306208,-0.172244,-0.029599,-0.331660,0.134525,0.258120,-0.613385,-0.328521
3,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219
4,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219


In [44]:
scaled_df['quality']=data['quality']
scaled_df['type']=data['type']
scaled_df['Calidad_2']=np.where(scaled_df['quality']<=5,0,1) 
scaled_df['White_wine']=np.where(scaled_df['type']=='white',1,0)
scaled_df['Red_wine']=np.where(scaled_df['type']=='red',1,0)
scaled_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,Calidad2,White_wine,Red_wine,Calidad_2
0,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558,6,white,1,1,0,1
1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615,6,white,1,1,0,1
2,0.682458,-0.362438,0.559966,0.306208,-0.172244,-0.029599,-0.331660,0.134525,0.258120,-0.613385,-0.328521,6,white,1,1,0,1
3,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219,6,white,1,1,0,1
4,-0.011808,-0.666161,0.009406,0.642523,0.056126,0.928254,1.243074,0.301278,-0.177272,-0.882212,-0.496219,6,white,1,1,0,1


In [46]:
train_white, test_white = train_test_split(scaled_df[scaled_df['type']=='white'], test_size=0.3)
train_red, test_red = train_test_split(scaled_df[scaled_df['type']=='red'], test_size=0.3)
print(len(train_white))
print(len(test_white))
print(len(train_red))
print(len(test_red))
print(len(scaled_df))

3428
1470
1119
480
6497


In [52]:
feature_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
X_W = train_white[feature_cols]

y_W = train_white['Calidad_2']

SVM_WW = SVC(kernel='linear')
SVM_WW.fit(X_W, y_W)
test_white['Pred_Calidad_2']=SVM_WW.predict(test_white[feature_cols])

In [55]:
print('Matriz de confusión:',confusion_matrix(test_white['Calidad_2'], test_white['Pred_Calidad_2']))
print('Accuracy del modelo:',accuracy_score(test_white['Calidad_2'], test_white['Pred_Calidad_2']))

Matriz de confusión: [[230 276]
 [ 94 870]]
Accuracy del modelo: 0.7482993197278912


In [60]:
feature_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
X_R = train_red[feature_cols]

y_R = train_red['Calidad_2']

SVM_R_W = SVC(kernel='linear')
SVM_R_W.fit(X_R, y_R)
test_red['Pred_Calidad_2']=SVM_R_W.predict(test_red[feature_cols])

In [61]:
print('Matriz de confusión:',confusion_matrix(test_red['Calidad_2'], test_red['Pred_Calidad_2']))
print('Accuracy del modelo:',accuracy_score(test_red['Calidad_2'], test_red['Pred_Calidad_2']))

Matriz de confusión: [[173  54]
 [ 68 185]]
Accuracy del modelo: 0.7458333333333333


# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


## Con el vino rojo los diferentes kernels

In [70]:
SVM_R_W = SVC(kernel='poly')
SVM_R_W.fit(X_R, y_R)
test_red['Pred_Calidad_2']=SVM_R_W.predict(test_red[feature_cols])
print('Matriz de confusión:',confusion_matrix(test_red['Calidad_2'], test_red['Pred_Calidad_2']))
print('Accuracy del modelo - Kernel poly:',accuracy_score(test_red['Calidad_2'], test_red['Pred_Calidad_2']))

Matriz de confusión: [[187  40]
 [ 69 184]]
Accuracy del modelo - Kernel poly: 0.7729166666666667


In [72]:
SVM_R_W = SVC(kernel='rbf')
SVM_R_W.fit(X_R, y_R)
test_red['Pred_Calidad_2']=SVM_R_W.predict(test_red[feature_cols])
print('Matriz de confusión:',confusion_matrix(test_red['Calidad_2'], test_red['Pred_Calidad_2']))
print('Accuracy del modelo - Kernel rbf:',accuracy_score(test_red['Calidad_2'], test_red['Pred_Calidad_2']))

Matriz de confusión: [[170  57]
 [ 60 193]]
Accuracy del modelo - Kernel rbf: 0.75625


In [75]:
SVM_R_W = SVC(kernel='sigmoid')
SVM_R_W.fit(X_R, y_R)
test_red['Pred_Calidad_2']=SVM_R_W.predict(test_red[feature_cols])
print('Matriz de confusión:',confusion_matrix(test_red['Calidad_2'], test_red['Pred_Calidad_2']))
print('Accuracy del modelo - Kernel  sigmoid:',accuracy_score(test_red['Calidad_2'], test_red['Pred_Calidad_2']))

Matriz de confusión: [[137  90]
 [ 97 156]]
Accuracy del modelo - Kernel  sigmoid: 0.6104166666666667


## Con el vino blanco los diferentes kernels

In [79]:
SVM_WW = SVC(kernel='poly')
SVM_WW.fit(X_W, y_W)
test_white['Pred_Calidad_2']=SVM_WW.predict(test_white[feature_cols])
print('Matriz de confusión:',confusion_matrix(test_white['Calidad_2'], test_white['Pred_Calidad_2']))
print('Accuracy del modelo - Kernel poly:',accuracy_score(test_white['Calidad_2'], test_white['Pred_Calidad_2']))

Matriz de confusión: [[158 348]
 [ 54 910]]
Accuracy del modelo - Kernel poly: 0.726530612244898


In [80]:
SVM_WW = SVC(kernel='rbf')
SVM_WW.fit(X_W, y_W)
test_white['Pred_Calidad_2']=SVM_WW.predict(test_white[feature_cols])
print('Matriz de confusión:',confusion_matrix(test_white['Calidad_2'], test_white['Pred_Calidad_2']))
print('Accuracy del modelo - Kernel rbf:',accuracy_score(test_white['Calidad_2'], test_white['Pred_Calidad_2']))

Matriz de confusión: [[285 221]
 [111 853]]
Accuracy del modelo - Kernel rbf: 0.7741496598639456


In [83]:
SVM_WW = SVC(kernel='sigmoid')
SVM_WW.fit(X_W, y_W)
test_white['Pred_Calidad_2']=SVM_WW.predict(test_white[feature_cols])
print('Matriz de confusión:',confusion_matrix(test_white['Calidad_2'], test_white['Pred_Calidad_2']))
print('Accuracy del modelo - Kernel sigmoid:',accuracy_score(test_white['Calidad_2'], test_white['Pred_Calidad_2']))

Matriz de confusión: [[219 287]
 [207 757]]
Accuracy del modelo - Kernel sigmoid: 0.6639455782312925


# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

Se realizarán dos ciclos para recorrer todas las posibilidades de los parámetros

In [89]:
salida = []
for gamma in [0.01, 0.001, 0.0001]:
    for C in [0.1, 1, 10, 100, 1000]:
        clf = SVC(C=C, gamma=gamma,kernel='rbf') 
        clf.fit(X_W, y_W)
        test_white['Pred_Calidad_2']=clf.predict(test_white[feature_cols]) 
        accuracy=accuracy_score(test_white['Calidad_2'], test_white['Pred_Calidad_2']) 
        salida.append((C, gamma, accuracy))

In [90]:
salida

[(0.1, 0.01, 0.6727891156462585),
 (1, 0.01, 0.7612244897959184),
 (10, 0.01, 0.7707482993197279),
 (100, 0.01, 0.7795918367346939),
 (1000, 0.01, 0.7707482993197279),
 (0.1, 0.001, 0.6557823129251701),
 (1, 0.001, 0.6768707482993197),
 (10, 0.001, 0.7435374149659864),
 (100, 0.001, 0.7612244897959184),
 (1000, 0.001, 0.7721088435374149),
 (0.1, 0.0001, 0.6557823129251701),
 (1, 0.0001, 0.6557823129251701),
 (10, 0.0001, 0.6775510204081633),
 (100, 0.0001, 0.7421768707482993),
 (1000, 0.0001, 0.7482993197278912)]

Parámetros C=100 Gamma=0.01 con accuracy de 77,9%

# Exercise 6.5

Compare the results with other methods

#### Regresión Logistica

In [92]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

logreg.fit(X_W, y_W)
test_white['Pred_Calidad_2']=logreg.predict(test_white[feature_cols])
print('Matriz de confusión - regresión logistica:',confusion_matrix(test_white['Calidad_2'], test_white['Pred_Calidad_2']))
print('Accuracy regresión logistica :',accuracy_score(test_white['Calidad_2'], test_white['Pred_Calidad_2']))

Matriz de confusión - regresión logistica: [[248 258]
 [102 862]]
Accuracy regresión logistica : 0.7551020408163265


Accuracy inferior al de la maquina

__________________________________________

# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

Entrenar el modelo de regresión lineal

In [93]:
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from math import sqrt

In [94]:
warnings.filterwarnings("ignore")

In [99]:
train, test = train_test_split(scaled_df, test_size=0.3)
linreg = LinearRegression()
X = train[feature_cols]
y = train['quality']
linreg.fit(X, y)
test['Pred_quality_wine']=linreg.predict(test[feature_cols])


Coeficientes

In [97]:
coef_lin=pd.DataFrame(np.transpose(linreg.coef_))
cols=pd.DataFrame(np.transpose(feature_cols))
coef=pd.concat([cols, coef_lin], axis=1, ignore_index=True)
coef

,0,1
0,fixed acidity,0.086537
1,volatile acidity,-0.235784
2,citric acid,-0.031451
3,residual sugar,0.209269
4,chlorides,-0.018627
5,free sulfur dioxide,0.089112
6,total sulfur dioxide,-0.132205
7,density,-0.158385
8,pH,0.062292
9,sulphates,0.118664


Las variables que explican el resultado son alcohol y residual sugar directamente proporcional al resultado y density y volatile acidity de manera inversamente proporcional

RMSE

In [100]:
MSE_=metrics.mean_squared_error(test['quality'], test['Pred_quality_wine'])
RMSE=sqrt(MSE_)
print('RMSE:', RMSE)

RMSE: 0.7339982808235654


# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [101]:
from sklearn.linear_model import Ridge

In [109]:
ridge_reg = Ridge(alpha=0.1)
ridge_reg.fit(X, y)
test['Pred_quality_wine'] = ridge_reg.predict(test[feature_cols])
print('RMSE-ridge alpha 0.1:',np.sqrt(metrics.mean_squared_error(test['quality'], test['Pred_quality_wine'])))
coef_ridge_01=pd.DataFrame(np.transpose(ridge_reg.coef_))

RMSE-ridge alpha 0.1: 0.7339979616912781


In [108]:
ridge_reg = Ridge(alpha=1)
ridge_reg.fit(X, y)
test['Pred_quality_wine'] = ridge_reg.predict(test[feature_cols])
print('RMSE-ridge alpha  1:',np.sqrt(metrics.mean_squared_error(test['quality'], test['Pred_quality_wine'])))
coef_ridge_1=pd.DataFrame(np.transpose(ridge_reg.coef_))

RMSE-ridge alpha  1: 0.7339951534023277


In [112]:
coef=pd.concat([cols,coef_lin, coef_ridge_01,coef_ridge_1], axis=1, ignore_index=True)
coef.columns=(['Columna','Coeficiente lineal','Coeficiente Ridge 0,1','Coeficiente Ridge 0,1'])
coef

,Columna,Coeficiente lineal,"Coeficiente Ridge 0,1","Coeficiente Ridge 0,1"
0,fixed acidity,0.086537,0.090593,0.090202
1,volatile acidity,-0.235784,-0.224120,-0.224116
2,citric acid,-0.031451,-0.024163,-0.024108
3,residual sugar,0.209269,0.216539,0.215998
4,chlorides,-0.018627,-0.020461,-0.020541
5,free sulfur dioxide,0.089112,0.097000,0.096939
6,total sulfur dioxide,-0.132205,-0.140177,-0.140034
7,density,-0.158385,-0.180114,-0.179374
8,pH,0.062292,0.056851,0.056623
9,sulphates,0.118664,0.118900,0.118800


Los coeficientes no sufresn cambios significativos

Los coeficientes de La regresión ridge de alpha 0.1 y alpha 1 al igual que con la regresión lineal son muy cercanos

# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [113]:
from sklearn.linear_model import Lasso

In [115]:
lasso_reg = Lasso(alpha=0.01)
lasso_reg.fit(X, y)
test['Pred_quality_wine'] = lasso_reg.predict(test[feature_cols])
print('RMSE-Lasso alpha 0,01:',np.sqrt(metrics.mean_squared_error(test['quality'], test['Pred_quality_wine'])))
coef_lasso_001=pd.DataFrame(np.transpose(lasso_reg.coef_))

RMSE-Lasso alpha 0,01: 0.7362052926995535


In [116]:
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
test['Pred_quality_wine'] = lasso_reg.predict(test[feature_cols])
print('RMSE-Lasso alpha 0,1:',np.sqrt(metrics.mean_squared_error(test['quality'], test['Pred_quality_wine'])))
coef_lasso_01=pd.DataFrame(np.transpose(lasso_reg.coef_))

RMSE-Lasso alpha 0,1: 0.7594418924222878


In [117]:
lasso_reg = Lasso(alpha=1)
lasso_reg.fit(X, y)
test['Pred_quality_wine'] = lasso_reg.predict(test[feature_cols])
print('RMSE-Lasso alpha 1:',np.sqrt(metrics.mean_squared_error(test['quality'], test['Pred_quality_wine'])))
coef_lasso_1=pd.DataFrame(np.transpose(lasso_reg.coef_))

RMSE-Lasso alpha 1: 0.8655736701296507


In [121]:
coef=pd.concat([cols,coef_lin, coef_lasso_001,coef_lasso_01,coef_lasso_1], axis=1, ignore_index=True)
coef.columns=(['Columna','Coeficiente lineal','Coeficiente Lasso 0,01','Coeficiente Lasso 0,1','Coeficiente Lasso 1'])
coef

,Columna,Coeficiente lineal,"Coeficiente Lasso 0,01","Coeficiente Lasso 0,1",Coeficiente Lasso 1
0,fixed acidity,0.086537,-0.000000,-0.000000,-0.0
1,volatile acidity,-0.235784,-0.223802,-0.133063,-0.0
2,citric acid,-0.031451,-0.006263,0.000000,0.0
3,residual sugar,0.209269,0.086521,0.000000,-0.0
4,chlorides,-0.018627,-0.023128,-0.000000,-0.0
5,free sulfur dioxide,0.089112,0.066537,0.000000,0.0
6,total sulfur dioxide,-0.132205,-0.086330,-0.000000,-0.0
7,density,-0.158385,-0.000000,-0.000000,-0.0
8,pH,0.062292,0.000000,0.000000,0.0
9,sulphates,0.118664,0.085351,0.000000,0.0


Los coeficientes disminuyen paulatinamente conforme aumenta el alpha

# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

In [ ]:
from sklearn.linear_model import LogisticRegression

In [124]:
log_reg = LogisticRegression()#solver='liblinear',C=1e9)

X = train[feature_cols]
y = train['Calidad_2']

log_reg.fit(X, y)
test['Pred_Calidad_2']=log_reg.predict(test[feature_cols])
print('Matriz de confusión regresión logistica:',confusion_matrix(test['Calidad_2'], test['Pred_Calidad_2']))
print('Accuracy regresión logistica:',accuracy_score(test['Calidad_2'], test['Pred_Calidad_2']))

Matriz de confusión regresión logistica: [[ 414  332]
 [ 195 1009]]
Accuracy regresión logistica: 0.7297435897435898


In [128]:
coef_log=pd.DataFrame(np.transpose(log_reg.coef_))
cols=pd.DataFrame(np.transpose(feature_cols))
coef_log=pd.concat([cols, coef_log], axis=1, ignore_index=True)
coef_log

,0,1
0,fixed acidity,0.104125
1,volatile acidity,-0.739415
2,citric acid,-0.116646
3,residual sugar,0.420976
4,chlorides,-0.026758
5,free sulfur dioxide,0.282022
6,total sulfur dioxide,-0.407331
7,density,-0.186772
8,pH,0.076708
9,sulphates,0.315205


Aumenta el coeficiente para el alcohol considerablemente.

In [129]:
from sklearn.metrics import f1_score
print('F1SCORE regresión logistica:',f1_score(test['Calidad_2'], test['Pred_Calidad_2']))

F1SCORE regresión logistica: 0.792927308447937


# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

In [150]:
X = train[feature_cols]
y = train['Calidad_2']

salida = []
coef_1 = cols##[]
colnames_1 = []
for C in [0.01, 0.1, 1]: ####Ciclo para recorrer los diferentes parametros c
    for penalty in ['l1', 'l2']: ###Ciclo para recorrer las diferentes penalidades
        log_reg = LogisticRegression(C=C, penalty=penalty,solver='liblinear')
        log_reg.fit(X, y)###Hacer el .fit
        test['Pred_Calidad_2']=log_reg.predict(test[feature_cols]) ##Predecir en la base test
        f1=f1_score(test['Calidad_2'], test['Pred_Calidad_2']) ##Obtener el f1score de la predicción
        salida.append((C, penalty, f1)) ###Guardar
        coef_tmp=pd.DataFrame(np.transpose(log_reg.coef_))
        coef_1=np.column_stack([coef_1, coef_tmp])

In [151]:
salida

[(0.01, 'l1', 0.7789142407553107),
 (0.01, 'l2', 0.7883556254917388),
 (0.1, 'l1', 0.7911915060951632),
 (0.1, 'l2', 0.7913555992141454),
 (1, 'l1', 0.7941869599371564),
 (1, 'l2', 0.792927308447937)]

In [152]:
coef_1=pd.DataFrame(coef_1)
coef_1.columns=(['Columna','C0.01-l1','C0.01-l2','C0.1-l1','C0.1-l2','C1-l1','C1-l2'])
coef_1

,Columna,C0.01-l1,C0.01-l2,C0.1-l1,C0.1-l2,C1-l1,C1-l2
0,fixed acidity,0,0.0636383,0,0.100147,0.0827543,0.104125
1,volatile acidity,-0.497103,-0.572549,-0.72518,-0.717018,-0.743202,-0.739415
2,citric acid,0,-0.045425,-0.0907778,-0.106846,-0.114673,-0.116646
3,residual sugar,0.0452129,0.295578,0.281338,0.406617,0.394874,0.420976
4,chlorides,0,-0.0701623,-0.0259923,-0.0327421,-0.0272979,-0.0267577
5,free sulfur dioxide,0,0.191752,0.237523,0.269466,0.278572,0.282022
6,total sulfur dioxide,0,-0.28066,-0.34249,-0.390289,-0.401129,-0.407331
7,density,0,-0.191914,0,-0.191099,-0.144173,-0.186772
8,pH,0,0.0660028,0.0137526,0.0764214,0.0640362,0.0767085
9,sulphates,0.0884986,0.255601,0.272632,0.308398,0.309249,0.315205
